<a href="https://colab.research.google.com/github/vkrisvasan/GenAIAgent/blob/main/AgentEx2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Building AI Agents With Crew AI using Google Gemini, Groq, LLama3
#CrewAI is an open-source framework that helps you build your own autonomous AI agents



In [1]:
pip install crewai langchain-google-genai crewai_tools

In [12]:
import os
import getpass
credential_names = ["GOOGLE_API_KEY","SERPER_API_KEY"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...SERPER_API_KEY··········


In [8]:
modelToBeused = "gemini-1.5-flash"

In [9]:
from crewai import Agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model=modelToBeused,verbose=True, temperature=0.5,google_api_key=os.getenv("GOOGLE_API_KEY"))

In [10]:
researcher= Agent(
role="Senior Researcher",
goal="""Uncover groundbreaking topics
in {topic}""",
backstory=("""Driven by curiosity, you are at the forefront of Innovation sharin knowledge and news that would change the world""")
,tools=[],llm=llm,allow_delegation=True,verbose=True,memory=True,)

In [11]:

writer= Agent(
role="Writer",
goal="Narrate compelling tech stories about {topic}",
backstory=("""Your expertise lies in breaking down complex topics into simpler a digestable knowledge, you specialize in
captivativing audience attention with yo writing and create engaging narratives and stories""")
,tools=[],llm=llm,allow_delegation=False,verbose=True,memory=True)